In [1]:
%env WORKDIR=/tmp/vault
%env VAULT_K8S_NAMESPACE=vault
%env VAULT_HELM_RELEASE_NAME=vault
%env VAULT_SERVICE_NAME=vault-internal 
%env K8S_CLUSTER_NAME=cluster.local 

env: WORKDIR=/tmp/vault
env: VAULT_K8S_NAMESPACE=vault
env: VAULT_HELM_RELEASE_NAME=vault
env: VAULT_SERVICE_NAME=vault-internal
env: K8S_CLUSTER_NAME=cluster.local


In [2]:
%%bash
cat > ${WORKDIR}/overrides_vso.yaml <<EOF
global:
   enabled: true
   tlsDisable: false # Disabling TLS to avoid issues when connecting to Vault via port forwarding

csi:
  enabled: false
  image:
    repository: hashicorp/vault-csi-provider


injector:
   enabled: false
   #repository: registry.connect.redhat.com/hashicorp/vault-k8s
   # Testing on Mac M1 so moving to arm image
   image:
      repository: docker.io/hashicorp/vault-k8s
      #repository: registry.connect.redhat.com/hashicorp/vault-k8s
      # Testing on Mac M1 so moving to arm image
   agentImage:
      repository: docker.io/hashicorp/vault
      #repository: registry.connect.redhat.com/hashicorp/vault
      # Testing on Mac M1 so moving to arm image

# Supported log levels include: trace, debug, info, warn, error
logLevel: "trace" # Set to trace for initial troubleshooting, info for normal operation

server:
# config.yaml
   image:
      repository: docker.io/hashicorp/vault-enterprise
      tag: 1.13.1-ent

   enterpriseLicense:
      secretName: vault-ent-license
   extraEnvironmentVars:
      VAULT_CACERT: /vault/userconfig/vault-ha-tls/vault.ca
      VAULT_TLSCERT: /vault/userconfig/vault-ha-tls/vault.crt
      VAULT_TLSKEY: /vault/userconfig/vault-ha-tls/vault.key
   volumes:
      - name: userconfig-vault-ha-tls
        secret:
         defaultMode: 420
         secretName: vault-ha-tls
   volumeMounts:
      - mountPath: /vault/userconfig/vault-ha-tls
        name: userconfig-vault-ha-tls
        readOnly: true
   standalone:
      enabled: false
   affinity: ""
   ha:
      enabled: true
      replicas: 3
      raft:
         enabled: true
         setNodeId: true
         config: |
            ui = true
            listener "tcp" {
               tls_disable = 0 # Disabling TLS to avoid issues when connecting to Vault via port forwarding
               address = "[::]:8200"
               cluster_address = "[::]:8201"
               tls_cert_file = "/vault/userconfig/vault-ha-tls/vault.crt"
               tls_key_file  = "/vault/userconfig/vault-ha-tls/vault.key"
               tls_client_ca_file = "/vault/userconfig/vault-ha-tls/vault.ca"
               api_address = "https://vault-active.vault.svc.cluster.local:8200"
            }
            storage "raft" {
               path = "/vault/data"
            
               retry_join {
                  auto_join             = "provider=k8s namespace=vault label_selector=\"component=server,app.kubernetes.io/name=vault\""
                  auto_join_scheme      = "https"
                  leader_ca_cert_file   = "/vault/userconfig/vault-ha-tls/vault.ca"
                  leader_tls_servername = "vaultpr-0.vault-internal" #Tiene que matchear una SAN del certificado
               }
            
            }
            disable_mlock = true
            service_registration "kubernetes" {}
EOF


In [3]:
%%bash
helm upgrade -n $VAULT_K8S_NAMESPACE $VAULT_HELM_RELEASE_NAME hashicorp/vault -f ${WORKDIR}/overrides_vso.yaml 

Release "vault" has been upgraded. Happy Helming!
NAME: vault
LAST DEPLOYED: Mon May 22 17:51:07 2023
NAMESPACE: vault
STATUS: deployed
REVISION: 2
NOTES:
Thank you for installing HashiCorp Vault!

Now that you have deployed Vault, you should look over the docs on using
Vault with Kubernetes available here:

https://www.vaultproject.io/docs/


Your release is named vault. To learn more about the release, try:

  $ helm status vault
  $ helm get manifest vault


In [4]:
%%bash
kubectl exec -n $VAULT_K8S_NAMESPACE vault-0 -- vault operator unseal $(jq -r ".unseal_keys_b64[]" ${WORKDIR}/cluster-keys.json)

Key                     Value
---                     -----
Seal Type               shamir
Initialized             true
Sealed                  false
Total Shares            1
Threshold               1
Version                 1.13.1+ent
Build Date              2023-03-23T20:09:57Z
Storage Type            raft
Cluster Name            vault-cluster-7156bb83
Cluster ID              1bdac7f0-bbcf-7efa-33c2-9f4fe15f5d9b
HA Enabled              true
HA Cluster              https://vault-0.vault-internal:8201
HA Mode                 active
Active Since            2023-05-22T15:50:03.887529469Z
Raft Committed Index    124
Raft Applied Index      124
Last WAL                33


In [5]:
%%bash
kubectl exec -n $VAULT_K8S_NAMESPACE vault-1 -- vault operator unseal $(jq -r ".unseal_keys_b64[]" ${WORKDIR}/cluster-keys.json)

Key                                    Value
---                                    -----
Seal Type                              shamir
Initialized                            true
Sealed                                 false
Total Shares                           1
Threshold                              1
Version                                1.13.1+ent
Build Date                             2023-03-23T20:09:57Z
Storage Type                           raft
Cluster Name                           vault-cluster-7156bb83
Cluster ID                             1bdac7f0-bbcf-7efa-33c2-9f4fe15f5d9b
HA Enabled                             true
HA Cluster                             https://vault-0.vault-internal:8201
HA Mode                                standby
Active Node Address                    https://10.244.0.4:8200
Performance Standby Node               true
Performance Standby Last Remote WAL    0
Raft Committed Index                   126
Raft Applied Index                     126


In [6]:
%%bash
kubectl exec -n $VAULT_K8S_NAMESPACE vault-2 -- vault operator unseal $(jq -r ".unseal_keys_b64[]" ${WORKDIR}/cluster-keys.json)

Key                                    Value
---                                    -----
Seal Type                              shamir
Initialized                            true
Sealed                                 false
Total Shares                           1
Threshold                              1
Version                                1.13.1+ent
Build Date                             2023-03-23T20:09:57Z
Storage Type                           raft
Cluster Name                           vault-cluster-7156bb83
Cluster ID                             1bdac7f0-bbcf-7efa-33c2-9f4fe15f5d9b
HA Enabled                             true
HA Cluster                             https://vault-0.vault-internal:8201
HA Mode                                standby
Active Node Address                    https://10.244.0.4:8200
Performance Standby Node               true
Performance Standby Last Remote WAL    0
Raft Committed Index                   128
Raft Applied Index                     128


In [7]:
%%bash
cat ${WORKDIR}/cluster-keys.json | jq -r ".root_token"

hvs.YngUaWsrp3hk4YtNMcJSz35I


In [8]:
%env VAULT_ADDR=https://127.0.0.1:8200
%env VAULT_TOKEN=hvs.YngUaWsrp3hk4YtNMcJSz35I
%env VAULT_CACERT=/tmp/vault/vault.ca

env: VAULT_ADDR=https://127.0.0.1:8200
env: VAULT_TOKEN=hvs.YngUaWsrp3hk4YtNMcJSz35I
env: VAULT_CACERT=/tmp/vault/vault.ca


In [9]:
%%bash
# Role associated to default service account in the app namespace
vault write auth/kubernetes/role/role1 \
    bound_service_account_names=default \
    bound_service_account_namespaces=vault \
    policies=dev \
    ttl=24h


Success! Data written to: auth/kubernetes/role/role1


In [10]:
%%bash
# Role associated to default service account in the app namespace
vault write auth/kubernetes/role/role2 \
    bound_service_account_names=default \
    bound_service_account_namespaces=default \
    policies=dev \
    ttl=24h

Success! Data written to: auth/kubernetes/role/role2


In [11]:
%%bash
vault policy write dev - <<EOF
path "kvv2/*" {
  capabilities = ["read"]
}
EOF


Success! Uploaded policy: dev


In [12]:
%%bash
vault secrets enable -path=kvv2 kv-v2

Success! Enabled the kv-v2 secrets engine at: kvv2/


In [13]:
%%bash
vault kv put kvv2/webapp/config username="static-user" password="static-password"

===== Secret Path =====
kvv2/data/webapp/config

======= Metadata =======
Key                Value
---                -----
created_time       2023-05-22T15:52:23.547810964Z
custom_metadata    <nil>
deletion_time      n/a
destroyed          false
version            1


In [15]:
%%bash
# Install Operator
helm search repo hashicorp/vault-secrets-operator --devel
helm install --namespace $VAULT_K8S_NAMESPACE vault-secrets-operator hashicorp/vault-secrets-operator --version 0.1.0-beta
kubectl get pods -n vault --watch


NAME                            	CHART VERSION	APP VERSION	DESCRIPTION                          
hashicorp/vault-secrets-operator	0.1.0-beta   	0.1.0-beta 	Official Vault Secrets Operator Chart
NAME: vault-secrets-operator
LAST DEPLOYED: Mon May 22 17:53:13 2023
NAMESPACE: vault
STATUS: deployed
REVISION: 1


In [16]:
%%bash
## VaultConnection Custom Resource
cat > vso_crd.yaml <<EOF
---
apiVersion: secrets.hashicorp.com/v1alpha1
kind: VaultConnection
metadata:
  namespace: vault
  name: example
spec:
  # required configuration
  # address to the Vault server.
  address: https://vault.vault.svc.cluster.local:8200

  # optional configuration
  # HTTP headers to be included in all Vault requests.
  # headers: []
  # TLS server name to use as the SNI host for TLS connections.
  # tlsServerName: ""
  # skip TLS verification for TLS connections to Vault.
  skipTLSVerify: true
  # the trusted PEM encoded CA certificate chain stored in a Kubernetes Secret
  # caCertSecretRef: ""

EOF
kubectl apply -f vso_crd.yaml


vaultconnection.secrets.hashicorp.com/example created


In [17]:
%%bash
## VaultConnection Custom Resource
cat > vso_crd_2.yaml <<EOF
---
apiVersion: secrets.hashicorp.com/v1alpha1
kind: VaultConnection
metadata:
  namespace: default
  name: example
spec:
  # required configuration
  # address to the Vault server.
  address: https://vault.vault.svc.cluster.local:8200

  # optional configuration
  # HTTP headers to be included in all Vault requests.
  # headers: []
  # TLS server name to use as the SNI host for TLS connections.
  # tlsServerName: ""
  # skip TLS verification for TLS connections to Vault.
  skipTLSVerify: true
  # the trusted PEM encoded CA certificate chain stored in a Kubernetes Secret
  # caCertSecretRef: ""

EOF
kubectl apply -f vso_crd_2.yaml

vaultconnection.secrets.hashicorp.com/example created


In [18]:
%%bash
cat > vaultauth_crd.yaml <<EOF
---
apiVersion: secrets.hashicorp.com/v1alpha1
kind: VaultAuth
metadata:
  namespace: vault
  name: example
spec:
  # required configuration
  # VaultConnectionRef of the corresponding VaultConnection CustomResource.
  # If no value is specified the Operator will default to the `default` VaultConnection,
  # configured in its own Kubernetes namespace.
  vaultConnectionRef: example
  # Method to use when authenticating to Vault.
  method: kubernetes
  # Mount to use when authenticating to auth method.
  mount: kubernetes
  # Kubernetes specific auth configuration, requires that the Method be set to kubernetes.
  kubernetes:
    # role to use when authenticating to Vault
    role: role1
    # ServiceAccount to use when authenticating to Vault
    # it is recommended to always provide a unique serviceAccount per Pod/application
    serviceAccount: default

  # optional configuration
  # Vault namespace where the auth backend is mounted (requires Vault Enterprise)
  # namespace: ""
  # Params to use when authenticating to Vault
  # params: []
  # HTTP headers to be included in all Vault authentication requests.
  # headers: []
EOF
kubectl apply -f vaultauth_crd.yaml

bash: line 1: default: command not found


vaultauth.secrets.hashicorp.com/example created


In [19]:
%%bash
cat > vaultauth_crd2.yaml <<EOF
---
apiVersion: secrets.hashicorp.com/v1alpha1
kind: VaultAuth
metadata:
  namespace: default
  name: example2
spec:
  # required configuration
  # VaultConnectionRef of the corresponding VaultConnection CustomResource.
  # If no value is specified the Operator will default to the `default` VaultConnection,
  # configured in its own Kubernetes namespace.
  vaultConnectionRef: example
  # Method to use when authenticating to Vault.
  method: kubernetes
  # Mount to use when authenticating to auth method.
  mount: kubernetes
  # Kubernetes specific auth configuration, requires that the Method be set to kubernetes.
  kubernetes:
    # role to use when authenticating to Vault
    role: role2
    # ServiceAccount to use when authenticating to Vault
    # it is recommended to always provide a unique serviceAccount per Pod/application
    serviceAccount: default

  # optional configuration
  # Vault namespace where the auth backend is mounted (requires Vault Enterprise)
  # namespace: ""
  # Params to use when authenticating to Vault
  # params: []
  # HTTP headers to be included in all Vault authentication requests.
  # headers: []
EOF
kubectl apply -f vaultauth_crd2.yaml


# VaultConnection <-- VaultAuth <-- VaultStaticSecret/VaultPKISecret/VaultDynamicSecret 

bash: line 1: default: command not found


vaultauth.secrets.hashicorp.com/example2 created


## CRD Static Secret

In [20]:
%%bash
## Support KVv1 and KVv2
cat > static_secret.yaml <<EOF
---
apiVersion: secrets.hashicorp.com/v1alpha1
kind: VaultStaticSecret
metadata:
  namespace: vault
  name: example
spec:
  vaultAuthRef: example
  mount: kvv2
  type: kv-v2
  name: webapp/config
  refreshAfter: 60s
  destination:
    create: true
    name: static-secret1
EOF
kubectl apply -f static_secret.yaml


vaultstaticsecret.secrets.hashicorp.com/example created


In [21]:
%%bash
## Support KVv1 and KVv2
cat > static_secret.yaml <<EOF
---
apiVersion: secrets.hashicorp.com/v1alpha1
kind: VaultStaticSecret
metadata:
  namespace: default
  name: example
spec:
  vaultAuthRef: example2
  mount: kvv2
  type: kv-v2
  name: webapp/config
  refreshAfter: 60s
  destination:
    create: true
    name: static-secret1
EOF
kubectl apply -f static_secret.yaml

vaultstaticsecret.secrets.hashicorp.com/example created


In [22]:
%%bash
kubectl get secrets -n vault

NAME                                           TYPE                 DATA   AGE
sh.helm.release.v1.vault-secrets-operator.v1   helm.sh/release.v1   1      60s
sh.helm.release.v1.vault.v1                    helm.sh/release.v1   1      5m14s
sh.helm.release.v1.vault.v2                    helm.sh/release.v1   1      3m6s
static-secret1                                 Opaque               3      6s
vault-ent-license                              Opaque               1      5m24s
vault-ha-tls                                   Opaque               3      5m38s
vso-cc-storage-hmac-key                        Opaque               1      48s


In [23]:
%%bash
kubectl get secret static-secret1 -n vault -o yaml

apiVersion: v1
data:
  _raw: eyJkYXRhIjp7InBhc3N3b3JkIjoic3RhdGljLXBhc3N3b3JkIiwidXNlcm5hbWUiOiJzdGF0aWMtdXNlciJ9LCJtZXRhZGF0YSI6eyJjcmVhdGVkX3RpbWUiOiIyMDIzLTA1LTIyVDE1OjUyOjIzLjU0NzgxMDk2NFoiLCJjdXN0b21fbWV0YWRhdGEiOm51bGwsImRlbGV0aW9uX3RpbWUiOiIwMDAxLTAxLTAxVDAwOjAwOjAwWiIsImRlc3Ryb3llZCI6ZmFsc2UsInZlcnNpb24iOjF9fQ==
  password: c3RhdGljLXBhc3N3b3Jk
  username: c3RhdGljLXVzZXI=
kind: Secret
metadata:
  creationTimestamp: "2023-05-22T15:54:07Z"
  labels:
    app.kubernetes.io/component: secret-sync
    app.kubernetes.io/managed-by: hashicorp-vso
    app.kubernetes.io/name: vault-secrets-operator
  name: static-secret1
  namespace: vault
  ownerReferences:
  - apiVersion: secrets.hashicorp.com/v1alpha1
    kind: VaultStaticSecret
    name: example
    uid: 8a347963-3aaa-4ced-9f7b-37e2161a2e24
  resourceVersion: "980"
  uid: 7c17d186-a279-48ef-87c8-e11268749f40
type: Opaque


In [24]:
%%bash
cat > mypod.yaml <<EOF
apiVersion: v1
kind: Pod
metadata:
  name: mypod
  namespace: vault
spec:
  containers:
  - name: mypod
    image: redis
    volumeMounts:
    - name: foo
      mountPath: "/etc/foo"
      readOnly: true
  volumes:
  - name: foo
    secret:
      secretName: static-secret1
      optional: true
EOF
kubectl apply -f mypod.yaml

# https://developer.hashicorp.com/vault/tutorials/kubernetes/vault-secrets-operator

pod/mypod created


In [25]:
%%bash 
sleep 10 
kubectl exec mypod -n vault -- cat /etc/foo/username
echo ""
echo "------"
kubectl exec mypod -n vault -- cat /etc/foo/password

static-user
------
static-password

In [28]:
%%bash
cat > mypod2.yaml <<EOF
apiVersion: v1
kind: Pod
metadata:
  name: mypod2
  namespace: vault
  # Using a different service account
spec:
  serviceAccountName: vault
  containers:
  - name: mypod
    image: redis
    volumeMounts:
    - name: foo
      mountPath: "/etc/foo"
      readOnly: true
  volumes:
  - name: foo
    secret:
      secretName: static-secret1
      optional: true
EOF
kubectl apply -f mypod2.yaml

pod/mypod2 created


In [30]:
%%bash 
sleep 10 
kubectl exec mypod2 -n vault -- cat /etc/foo/username
echo ""
echo "------"
kubectl exec mypod2 -n vault -- cat /etc/foo/password

static-user
------
static-password

In [24]:
%%bash 
### For namespace prueba


kubectl delete VaultConnection vault-conn -n prueba
kubectl delete VaultStaticSecret example -n prueba
kubectl delete VaultAuth vault-auth-prueba -n prueba
kubectl delete ns prueba

kubectl create ns prueba

cat <<EOF | kubectl create -f -
---
apiVersion: secrets.hashicorp.com/v1alpha1
kind: VaultConnection
metadata:
  namespace: prueba
  name: vault-conn
spec:
  # required configuration
  # address to the Vault server.
  address: https://vault.vault.svc.cluster.local:8200

  # optional configuration
  # HTTP headers to be included in all Vault requests.
  # headers: []
  # TLS server name to use as the SNI host for TLS connections.
  # tlsServerName: ""
  # skip TLS verification for TLS connections to Vault.
  skipTLSVerify: true
  # the trusted PEM encoded CA certificate chain stored in a Kubernetes Secret
  # caCertSecretRef: ""
EOF

cat  <<EOF | kubectl create -f -
---
apiVersion: secrets.hashicorp.com/v1alpha1
kind: VaultAuth
metadata:
  namespace: prueba
  name: vault-auth-prueba
spec:
  # required configuration
  # VaultConnectionRef of the corresponding VaultConnection CustomResource.
  # If no value is specified the Operator will default to the `default` VaultConnection,
  # configured in its own Kubernetes namespace.
  vaultConnectionRef: vault-conn
  # Method to use when authenticating to Vault.
  method: kubernetes
  # Mount to use when authenticating to auth method.
  mount: kubernetes
  # Kubernetes specific auth configuration, requires that the Method be set to kubernetes.
  kubernetes:
    # role to use when authenticating to Vault
    role: role3
    # ServiceAccount to use when authenticating to Vault
    # it is recommended to always provide a unique serviceAccount per Pod/application
    serviceAccount: default

  # optional configuration
  # Vault namespace where the auth backend is mounted (requires Vault Enterprise)
  # namespace: ""
  # Params to use when authenticating to Vault
  # params: []
  # HTTP headers to be included in all Vault authentication requests.
  # headers: []
EOF

cat  <<EOF | kubectl create -f -
---
apiVersion: secrets.hashicorp.com/v1alpha1
kind: VaultStaticSecret
metadata:
  namespace: prueba
  name: example
spec:
  vaultAuthRef: vault-auth-prueba
  mount: kvv2
  type: kv-v2
  name: webapp/config
  refreshAfter: 60s
  destination:
    create: true
    name: static-secret1
EOF


vaultconnection.secrets.hashicorp.com "vault-conn" deleted
vaultstaticsecret.secrets.hashicorp.com "example" deleted
vaultauth.secrets.hashicorp.com "vault-auth-prueba" deleted
namespace "prueba" deleted
namespace/prueba created
vaultconnection.secrets.hashicorp.com/vault-conn created


bash: line 34: default: command not found


vaultauth.secrets.hashicorp.com/vault-auth-prueba created
vaultstaticsecret.secrets.hashicorp.com/example created


In [22]:
%%bash
# Role associated to default service account in the app namespace
vault write auth/kubernetes/role/role3 \
    bound_service_account_names=default \
    bound_service_account_namespaces=prueba \
    policies=dev \
    ttl=24h

Success! Data written to: auth/kubernetes/role/role3


In [27]:
%%bash
vault kv put kvv2/webapp user="user" pass="password"

== Secret Path ==
kvv2/data/webapp

======= Metadata =======
Key                Value
---                -----
created_time       2023-04-20T16:49:44.244149459Z
custom_metadata    <nil>
deletion_time      n/a
destroyed          false
version            1


In [26]:
%%bash
# Role associated to default service account in the app namespace
vault write auth/kubernetes/role/role4 \
    bound_service_account_names=vault-sa \
    bound_service_account_namespaces=test \
    policies=dev \
    ttl=24h

### For namespace test
kubectl delete VaultConnection vault-conn -n test
kubectl delete VaultStaticSecret example -n test
kubectl delete VaultAuth vault-auth-prueba -n test
kubectl delete sa vault-sa -n test
kubectl delete ns test

kubectl create ns test
kubectl create sa vault-sa -n test

cat <<EOF | kubectl create -f -
---
apiVersion: secrets.hashicorp.com/v1alpha1
kind: VaultConnection
metadata:
  namespace: test
  name: vault-conn
spec:
  # required configuration
  # address to the Vault server.
  address: https://vault.vault.svc.cluster.local:8200

  # optional configuration
  # HTTP headers to be included in all Vault requests.
  # headers: []
  # TLS server name to use as the SNI host for TLS connections.
  # tlsServerName: ""
  # skip TLS verification for TLS connections to Vault.
  skipTLSVerify: true
  # the trusted PEM encoded CA certificate chain stored in a Kubernetes Secret
  # caCertSecretRef: ""
EOF

cat  <<EOF | kubectl create -f -
---
apiVersion: secrets.hashicorp.com/v1alpha1
kind: VaultAuth
metadata:
  namespace: test
  name: vault-auth-prueba
spec:
  # required configuration
  # VaultConnectionRef of the corresponding VaultConnection CustomResource.
  # If no value is specified the Operator will default to the `default` VaultConnection,
  # configured in its own Kubernetes namespace.
  vaultConnectionRef: vault-conn
  # Method to use when authenticating to Vault.
  method: kubernetes
  # Mount to use when authenticating to auth method.
  mount: kubernetes
  # Kubernetes specific auth configuration, requires that the Method be set to kubernetes.
  kubernetes:
    # role to use when authenticating to Vault
    role: role4
    # ServiceAccount to use when authenticating to Vault
    # it is recommended to always provide a unique serviceAccount per Pod/application
    serviceAccount: vault-sa

  # optional configuration
  # Vault namespace where the auth backend is mounted (requires Vault Enterprise)
  # namespace: ""
  # Params to use when authenticating to Vault
  # params: []
  # HTTP headers to be included in all Vault authentication requests.
  # headers: []
EOF

cat  <<EOF | kubectl create -f -
---
apiVersion: secrets.hashicorp.com/v1alpha1
kind: VaultStaticSecret
metadata:
  namespace: test
  name: vault-secret-app100
spec:
  vaultAuthRef: vault-auth-prueba
  mount: kvv2
  type: kv-v2
  name: webapp/config
  refreshAfter: 60s
  destination:
    create: true
    name: k8s-secret-app100
EOF
 

Success! Data written to: auth/kubernetes/role/role4
vaultconnection.secrets.hashicorp.com "vault-conn" deleted


Error from server (NotFound): vaultstaticsecrets.secrets.hashicorp.com "example" not found


vaultauth.secrets.hashicorp.com "vault-auth-prueba" deleted
serviceaccount "vault-sa" deleted
namespace "test" deleted
namespace/test created
serviceaccount/vault-sa created
vaultconnection.secrets.hashicorp.com/vault-conn created


bash: line 41: default: command not found


vaultauth.secrets.hashicorp.com/vault-auth-prueba created
vaultstaticsecret.secrets.hashicorp.com/vault-secret-app100 created


In [28]:
%%bash
cat  <<EOF | kubectl create -f -
---
apiVersion: secrets.hashicorp.com/v1alpha1
kind: VaultStaticSecret
metadata:
  namespace: test
  name: vault-secret-app200
spec:
  vaultAuthRef: vault-auth-prueba
  mount: kvv2
  type: kv-v2
  name: webapp
  refreshAfter: 60s
  destination:
    create: true
    name: k8s-secret-app200
EOF

vaultstaticsecret.secrets.hashicorp.com/vault-secret-app200 created


In [29]:
%%bash
cat > mypod2_test.yaml <<EOF
apiVersion: v1
kind: Pod
metadata:
  name: mypod2
  namespace: test
  # Using a different service account
spec:
  containers:
  - name: mypod
    image: redis
    volumeMounts:
    - name: foo
      mountPath: "/etc/foo"
      readOnly: true
  volumes:
  - name: foo
    secret:
      secretName: k8s-secret-app200
      optional: true
EOF
kubectl apply -f mypod2_test.yaml

pod/mypod2 created


In [30]:
%%bash
# Role associated to default service account in the app namespace
vault write auth/kubernetes/role/role5 \
    bound_service_account_names=vault-sa \
    bound_service_account_namespaces=test \
    policies=dev \
    ttl=1h

### For namespace test
kubectl delete VaultStaticSecret vault-secret-app300 -n test
kubectl delete VaultAuth vault-auth-prueba2 -n test



cat  <<EOF | kubectl create -f -
---
apiVersion: secrets.hashicorp.com/v1alpha1
kind: VaultAuth
metadata:
  namespace: test
  name: vault-auth-prueba2
spec:
  # required configuration
  # VaultConnectionRef of the corresponding VaultConnection CustomResource.
  # If no value is specified the Operator will default to the `default` VaultConnection,
  # configured in its own Kubernetes namespace.
  vaultConnectionRef: vault-conn
  # Method to use when authenticating to Vault.
  method: kubernetes
  # Mount to use when authenticating to auth method.
  mount: kubernetes
  # Kubernetes specific auth configuration, requires that the Method be set to kubernetes.
  kubernetes:
    # role to use when authenticating to Vault
    role: role5
    # ServiceAccount to use when authenticating to Vault
    # it is recommended to always provide a unique serviceAccount per Pod/application
    serviceAccount: vault-sa

  # optional configuration
  # Vault namespace where the auth backend is mounted (requires Vault Enterprise)
  # namespace: ""
  # Params to use when authenticating to Vault
  # params: []
  # HTTP headers to be included in all Vault authentication requests.
  # headers: []
EOF

cat  <<EOF | kubectl create -f -
---
apiVersion: secrets.hashicorp.com/v1alpha1
kind: VaultStaticSecret
metadata:
  namespace: test
  name: vault-secret-app300
spec:
  vaultAuthRef: vault-auth-prueba
  mount: kvv2
  type: kv-v2
  name: webapp/config
  refreshAfter: 600s
  destination:
    create: true
    name: k8s-secret-app300
EOF
 

Success! Data written to: auth/kubernetes/role/role5


Error from server (NotFound): vaultstaticsecrets.secrets.hashicorp.com "vault-secret-app300" not found
Error from server (NotFound): vaultauths.secrets.hashicorp.com "vault-auth-prueba2" not found
bash: line 14: default: command not found


vaultauth.secrets.hashicorp.com/vault-auth-prueba2 created
vaultstaticsecret.secrets.hashicorp.com/vault-secret-app300 created
